search jupyter notebooks using llama-gpt?

search llama?


ask llm which function matches?

ask llm matching function?


In [ ]:
const {askLlamaAboutFunctions} = importer.import('ask llm about functions')
const {askLlamaAboutCategories} = importer.import('ask llm about categories')
const lookupCell = importer.lookupCell
const { cacheCells } = importer.import('cache notebook')
const { functionCache } = importer.import('cache rpc functions with llm descriptions')
const { cellCache } = importer.import('cell cache')
const {storeAllLlamaFunctions} = importer.import('store all notebook llm functions')

async function askLlamaMatchingFunction(query) {
  await storeAllLlamaFunctions()

  let filterCategories = await askLlamaAboutCategories(query)
  if(filterCategories.length > 0) {
    console.log(filterCategories)
  }

  let matchingFunction = []
  let functions = []
  let descriptions = []

  for(let i = 0; i < cellCache.length; i++) {
    let cell = cellCache[i]
    if(!cell[2].questions || !cell[2].questions[0]) continue
    let code = lookupCell(cell[1], cacheCells)

    if(!functionCache[cell[1]].exports) continue
    if(!functionCache[cell[1]].exports[0]) continue

    // limit number of calls based on filter
    if(filterCategories.length > 0 && !filterCategories.includes(code.filename)) {
      continue
    }

    // finally append to array that will be used to find the function
    functions[functions.length] = functionCache[cell[1]].exports[0]
    descriptions[descriptions.length] = functionCache[cell[1]].summary

    if(functions.length == 20) {
        let result = await askLlamaAboutFunctions(query, functions, descriptions)
        descriptions = []
        functions = []
        if(result)
          matchingFunction = matchingFunction.concat(result)
    }
  }
  if(functions.length > 0) {
      let result = await askLlamaAboutFunctions(query, functions, descriptions)
      descriptions = []
      functions = []
      if(result)
        matchingFunction = matchingFunction.concat(result)
  }
  return matchingFunction
}

module.exports = {
  askLlamaMatchingFunction
}



ask llm about categories?


In [ ]:
const { functionCache } = importer.import('cache rpc functions with llm descriptions')
const {askLlamaAboutFunctions} = importer.import('ask llm about functions')


async function askLlamaAboutCategories(query) {
  // TODO: list all categories in database
  let keys = Object.keys(functionCache)
  let categorys = keys.map(k => functionCache[k].categories)
    .concat(keys.map(k => functionCache[k].category))
    .filter((a, i, arr) => a && arr.indexOf(a) == i && !a.includes('\n'))

  // TODO: ask llm if any of the categories match, don't choose best one, choose all matches
  let returnValues = []
  let functions = []
  for(let i = 0; i < categorys.length; i++) {
    let category = categorys[i]
    functions[functions.length] = category

    if(functions.length == 20) {
      let result = await askLlamaAboutFunctions(query, functions, [], true)
      functions = []
      if(result)
        returnValues = returnValues.concat(result)
    }
  }
  if(functions.length > 0) {
      let result = await askLlamaAboutFunctions(query, functions, [], true)
      functions = []
      if(result)
        returnValues = returnValues.concat(result)
  }

  // TODO: return notebook filenames that contain matching categories
  let matching = keys.filter(k => returnValues.includes(functionCache[k].category) || returnValues.includes(functionCache[k].categories))
    .map(k => k.replace(/\[[0-9]*\]/, ''))
    .filter((a, i, arr) => a && arr.indexOf(a) == i)
  return matching
}

module.exports = {
  askLlamaAboutCategories
}



ask llm to generalize categories?


In [ ]:
const { functionCache } = importer.import('cache rpc functions with llm descriptions')
const { askLlamaAboutFunctions } = importer.import('ask llm about functions')
const { storeLlamaFunction } = importer.import('store llama function')
const { askLlamaToGeneralizeAll } = importer.import('ask llm for a shorter list of categories')

async function askLlamaGeneralizeCategories(categories) {
  let update = false
  let keys = Object.keys(functionCache)
  if(!categories) {
    categories = keys.map(k => functionCache[k].categories)
      .filter((a, i, arr) => a && arr.indexOf(a) == i && !a.includes('\n'))
    update = true
  }

  if(categories.length == 0) {
    return []
  }

  let uniqueValues = await askLlamaToGeneralizeAll(categories)
  console.log(uniqueValues)

  if(uniqueValues.length > 100) {
    uniqueValues = (await askLlamaGeneralizeCategories(uniqueValues))
      .filter((a, i, arr) => arr.indexOf(a) == i)
  }

  if(update) {
    let convertedList = []
    for(let i = 0; i < keys.length; i++) {
      let newCategory = await askLlamaAboutFunctions(functionCache[keys[i]].categories, uniqueValues, [], true)
      if(newCategory) {
        storeLlamaFunction(keys[i], functionCache[keys[i]].mtime, 
          functionCache[keys[i]].exports, functionCache[keys[i]].description, 
          functionCache[keys[i]].summary, functionCache[keys[i]].categories, newCategory)
      }
      convertedList[i] = newCategory
    }
    console.log(convertedList)
  }

  return uniqueValues
}

module.exports = {
  askLlamaGeneralizeCategories
}



ask llm for a shorter list of categories?


In [ ]:

async function askLlamaToGeneralize(categories) {
  const {llmPrompt} = await importer.import('create llm session')
  let q1 = 'Generalize this list of categories into a shorter list:\n' 
    + categories.join('\n') + '\nOnly return the list and nothing else.'
  console.log("User: " + q1);
  const a1 = await llmPrompt(q1);
  console.log("AI: " + a1);

  return a1.split(/\s*\n\s*|,\s*|\s*- |\s*\* /gi)
    .filter((a, i, arr) => a.trim().length > 0 /*&& arr.indexOf(a) == i*/)
}

async function askLlamaToGeneralizeAll(categories, groupSize = 10) {
  let resultValues = []
  let functions = []
  for(let i = 0; i < categories.length; i++) {
    let category = categories[i]
    functions[functions.length] = category

    if(functions.length == groupSize) {
      let result = await askLlamaToGeneralize(functions)
      functions = []
      resultValues = resultValues.concat(result)
    }
  }
  if(functions.length > 0) {
    let result = await askLlamaToGeneralize(functions)
    functions = []
    resultValues = resultValues.concat(result)
  }

  // TODO: if the list is still longer than say 100, call this function recursively until its short, haha
  let uniqueValues = resultValues.filter((a, i, arr) => arr.indexOf(a) == i)
  return uniqueValues
}

module.exports = {
  askLlamaToGeneralizeAll,
  askLlamaToGeneralize
}




ask llm about functions?



In [ ]:

async function askLlamaAboutFunctions(query, functions, descriptions, categories = false) {
  const {llmPrompt} = await importer.import('create llm session')

  let q1 = "Given a list of " + (categories ? 'categories' : "functions") + ":\n";
  for(let i = 0; i < functions.length; i++) {
      if(descriptions[i]) {
          q1 += functions[i] + ' - ' + descriptions[i] + '\n'
      } else {
          q1 += functions[i] + '\n'
      }
  }
  q1 += 'which one most closely matches the query \"' + query + '\"?'
  console.log("User: " + q1);
  const a1 = await llmPrompt(q1);
  console.log("AI: " + a1);
  // TODO: parse function name
  let result = a1.trim().split(/["`*'\n]/gi).filter(x => functions.indexOf(x) > -1)[0]
  return result
}

module.exports = {
  askLlamaAboutFunctions,
}




ask llm about code?


In [ ]:


async function askLlamaAboutCode(code) {
  const {llmPrompt} = await importer.import('create llm session')

  const q2 = "Give me a short breakdown of this code:\n" + code.substr(0, 2048) + "\nDocumentation only, discard any friendly remarks.";
  console.log("User: " + q2);

  const a2 = await llmPrompt(q2);
  console.log("AI: " + a2);

  return a2.trim()
}

module.exports = {
  askLlamaAboutCode,
}



ask llm about notebooks?



In [ ]:

const path = require('path')
const {listInProject} = importer.import('list project files');
const {askLlamaAboutFunctions} = importer.import('ask llm about functions')

function findNotebooks(dirname) {
    return listInProject(dirname, '{,*,*/,*/*/*,*/*/*/*}*.ipynb')
}

async function askLlamaAboutNotebooks(query) {
  let notebooks = findNotebooks(path.resolve(__dirname, '../'))
  let functions = []
  let descriptions = []

  for(let i = 0; i < notebooks.length; i++) {
    let notebookName = path.basename(notebooks[i]).replace('.ipynb', '')
    let notebookCamel = notebookName.substring(0, 1).toLocaleUpperCase() + notebookName.substring(1)
    let folderName = path.basename(path.dirname(notebooks[i]))
    let folderCamel = folderName.substring(0, 1).toLocaleUpperCase() + folderName.substring(1)

    functions[functions.length] = 'search' + folderCamel.replace(/[^a-z0-9]/gi, '') + notebookCamel.replace(/[^a-z0-9]/gi, '').substring(0, 15)
    descriptions[descriptions.length] = folderCamel + ' ' + notebookCamel

    // TODO: convert this to something like function.length + description.length + 4 < 2048
    if(functions.length == 20) {
      let result = await askLlamaAboutFunctions(query, functions, descriptions)
      if(result) {
        descriptions = [descriptions[functions.indexOf(result)]]
        functions = [result]
      } else {
        descriptions = []
        functions = []
      }
    }
  }
  if(functions.length > 1) {
    let result = await askLlamaAboutFunctions(query, functions, descriptions)
    if(result) functions = [result]
    else functions = []
  }

  return functions
}


module.exports = {
  askLlamaAboutNotebooks,
}




ask llm to summerize?


In [ ]:

async function askLlamaToSummerize(query) {
  const {llmPrompt} = await importer.import('create llm session')
  let q1 = 'Summerize this into one or two sentences:\n' + query
  console.log("User: " + q1);
  const a1 = await llmPrompt(q1);
  console.log("AI: " + a1);

  return a1.trim()
}

async function askLlamaToGeneralize(query) {
  const {llmPrompt} = await importer.import('create llm session')
  let q2 = 'How would you categorize this in two or three words:\n' + query + '\nReturn only the category.'
  console.log("User: " + q2);
  const a2 = await llmPrompt(q2);
  console.log("AI: " + a2);

  return a2.trim()
}



module.exports = {
  askLlamaToSummerize,
  askLlamaToGeneralize
}





store llama function?




In [ ]:
const { functionCache } = importer.import('cache rpc functions with llm descriptions')
const { updateCode } = importer.import('update code cell')

function storeLlamaFunction (cellId, mtime, exports, description, summary, categories, category) {
  functionCache[cellId] = {
    mtime,
    exports,
    description,
    summary,
    categories,
    category
  }
  var code = `
// cell cache automatically replaced
var functionCache = ${JSON.stringify(functionCache, null, 4)}

module.exports = {
  functionCache
}
`
  var cacheCell = importer.interpret('cache rpc functions with llm descriptions')
  updateCode(cacheCell, code)
}

module.exports = {
  storeLlamaFunction
}



store all notebook llm functions?


In [ ]:
const {askLlamaAboutCode} = importer.import('ask llm about code')
const {askLlamaToSummerize, askLlamaToGeneralize} = importer.import('ask llm to summerize')
const {getExports, cacheCells} = importer.import([
  'select code tree', 'get exports from source', 'cache notebook', 'cache all'])
const { functionCache } = importer.import('cache rpc functions with llm descriptions')
const { storeLlamaFunction } = importer.import('store llama function')

async function storeAllLlamaFunctions() {
  const getParameters = await importer.import('get c parameters')
  const pythonParams = await importer.import('python params in antlr')
  let cellCache = importer.import('cell cache').cellCache
  for(let i = 0; i < cellCache.length; i++) {
    let cell = cellCache[i]
    //if(!cell[2].questions || !cell[2].questions[0]) continue
    let code = importer.lookupCell(cell[1], cacheCells)
    if(code.code.trim().length == 0) {
      storeLlamaFunction(cell[1], code.mtime, [], '', '', '', '')
      continue
    }

    let summary
    let shortened
    let rpcFunction
    let categories
    let category
    let fresh = false

    if(typeof functionCache[cell[1]] != 'undefined') {
      if(
        // notebook hasn't changed
        code.mtime <= functionCache[cell[1]].mtime
        // don't both updating cache notebooks, 
        // otherwise this will run every time any notebook changes
        || code.filename.match(/cache/gi)
      ) {
        summary = functionCache[cell[1]].description
        shortened = functionCache[cell[1]].summary
        rpcFunction = functionCache[cell[1]].exports
      }

      categories = functionCache[cell[1]].categories
      category = functionCache[cell[1]].categories
    } else {
      fresh = true
    }

    // needs cleanup
    if(!summary || !categories || (categories + '').includes('\n')
      || summary.length < 256 || summary.match(/Find the derivative/gi) 
      || shortened.match(/Find the derivative/gi)
      || categories.match(/Code analysis request/gi)) {
      // TODO: this should cause the erroneous cell to show up every time and for these to be fixed next pass
      summary = await askLlamaAboutCode(code.code)
      shortened = await askLlamaToSummerize(summary)
      categories = await askLlamaToGeneralize(summary)
      fresh = true
    }
    if(typeof rpcFunction == 'undefined') {
      try {
        if(code.language == 'javascript')
          rpcFunction = getExports(code.code)
        if(code.language == 'c' || code.language == 'cpp')
          rpcFunction = (await getParameters(code.code)).map(p => typeof p == 'string' ? p : p[0])
        if(code.language == 'python') {
          const params = await pythonParams(code.code)
          rpcFunction = typeof params.function != 'undefined' ? [params.function] : params.map(p => p.function)
        }
        fresh = true
      } catch (e) {
        rpcFunction = []
      }
    }

    if(fresh) {
      // TODO: insert rpc function into sqlite database to make subsequent lookups faster
      storeLlamaFunction(cell[1], code.mtime, rpcFunction, summary, shortened, categories, category)
    }
  }
}

module.exports = {
  storeAllLlamaFunctions
}


create llm session?




In [ ]:
import path from "path"
import {getLlama, LlamaChatSession} from "node-llama-cpp"
import process from "process"

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
//const __dirname = path.dirname(fileURLToPath(import.meta.url));

let llama
let model
let context
let session
let initialChatHistory


async function initSession(context2, prompt) {
  if(!context2) {
    context = await model.createContext()
  } else {
    context = context2
  }

  session = new LlamaChatSession({
      contextSequence: context.getSequence(),
      systemPrompt: prompt ? prompt : '```markdown\n# You are a\n##large language model\nnamed Llama that provides clear and concise answers in beautifully crafted `markdown` unless otherwise instructed.\n</think>\n...```\n'
  })
  // initialize the model
  //console.log(await session.prompt())
  initialChatHistory = session.getChatHistory();

  // Reset the chat history
  session.setChatHistory(initialChatHistory);
  return session
}

async function createSession(modelPath) {
  if(!llama) {
    llama = await getLlama();
  }

  // TODO: customizable model here

  if(!model) {
      model = await llama.loadModel({
          // "code-llama-7b-chat.gguf"
          // "Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf"
          // 'ggml-model-f16.gguf'
          //modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'ggml-model-f16.gguf' ),
          //modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'DeepSeek-R1-Distill-Llama-8B-Q6_K.gguf' ),
          //modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'gemma-2-9b-it-Q6_K-Q8.gguf' ),
          //modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'deepseek-llm-7b-chat.Q5_K_M.gguf' ),
          //modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'Meta-Llama-3.1-70B-Instruct-Q5_K_S.gguf' ),
          modelPath: process.env.MODELPATH || modelPath || path.join(HOMEPATH, "llama.cpp", "models", 'Meta-Llama-3.1-8B-Instruct-Q6_K_L.gguf' ),
          //modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'deepseek-llm-7b-chat.Q6_K.gguf' ),
          contextSize: 2048
      })
  }

  await initSession()

  return session
}

async function getSession(model) {
  if(!session) {
    await createSession(model)
  }
  return session
}

async function llmPrompt(prompt, model) {
  if(!model || typeof model == 'string') {
    // specify model with the incoming session
    model = await getSession(model)
  }
  let result = await model.prompt(prompt, {
    temperature: 0.8,
    maxTokens: context.contextSize,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  // special fix for deepseek R1 distill
  if(result.match(/Find the derivative/gi)) {
    console.log('Response error', result)
    await initSession()
    result = await session.prompt(prompt, {
      temperature: 0.8,
      maxTokens: context.contextSize,
      onTextChunk: function (text) {
        process.stdout.write(text)
      }
    })
  }
  //if(result.includes('</think>')) {
  //  result = result.split('</think>\n')[1]
  //}
  if(result.startsWith('```') && result.endsWith('````')) {
    result = result.replace(/^```(markdown)*\n*|\n*```$/gi, '')
  }
  if(session == model)
    model.setChatHistory(initialChatHistory);
  return result
}

export default {
  llmPrompt,
  createSession,
  initSession,
  getSession,
}
